# Creating an AI-Powered Web Page Summarizer 

In [ ]:
import os 
import requests 
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
# The environment variables are loaded previously in a file called .env 

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key: 
    print("No API key was found!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but in a wrong format!")
elif api_key.strip() != api_key:
    print("An API key was found, but with space at the start or end of it!")
else:
    print("A correct API key was found!")

In [ ]:
openai = OpenAI()

In [ ]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}    

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


In [ ]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
def user_prompt(x):
    user_prompt = f"You are looking at a website titled {x.title}"
    user_prompt += "\nThe contents of this website is as follows; \
Please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too. \n\n"
    user_prompt += x.text
    return user_prompt

In [ ]:
def message_for(y):
    return[
        {"role":"system", "content": system_prompt},
        {"role":"user", "content": user_prompt(y)}
    ]

In [ ]:
def summarize(url):
    golabi = Website(url)
    response = openai.chat.completions.create(model = "gpt-4o-mini", messages = message_for(golabi))
    return response.choices[0].message.content

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://edition.cnn.com/")